In [ ]:
from configs.model_config import ModelPathConfig
!git clone https: // github.com/kk-digital/kcg-ml-sd1p4
%cd kcg-ml-sd1p4

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python3./download_models.py

In [ ]:
!python3./process_models.py

In [4]:

import os
import sys
import shutil
from os.path import join

base_directory = "../"
sys.path.insert(0, base_directory)

output_base_dir = join(base_directory, "./output/sd2-notebook/")
output_directory = join(output_base_dir, "clip_image_encoder/")

try:
    shutil.rmtree(output_directory)
except Exception as e:
    print(e, "\n", "Creating the path...")
    os.makedirs(output_directory, exist_ok=True)
else:
    os.makedirs(output_directory, exist_ok=True)

from stable_diffusion.stable_diffusion import StableDiffusion
from stable_diffusion.model.clip_image_encoder import CLIPImageEncoder
from stable_diffusion.utils_model import *
from stable_diffusion.utils_backend import *

In [ ]:
# Setup config (paths and such)
batch_size = 1
base_dir = os.getcwd()
config = ModelPathConfig(root_directory=base_dir)
config.create_directory_tree_folders()

In [ ]:
# Initialize clip
clip_text_embedder = CLIPTextEmbedder(device=get_device())
clip_text_embedder.load_submodels(tokenizer_path=config.get_model("clip/txt_emb_tokenizer"),
                                  transformer_path=config.get_model("clip/txt_emb_transformer"))
get_memory_status(device=get_device())

In [7]:
# get the text embeddings
null_prompt = torch.tensor(()).to(get_device())
embeddings = clip_text_embedder.forward("A computer virus dancing tango.")
get_memory_status(device=get_device())

In [ ]:
# check their shape
null_prompt.shape, embeddings.shape

In [9]:
# Cleanup clip
del clip_text_embedder

In [10]:
# save them to disk
torch.save(embeddings, join(output_directory, "embeddings.pt"))
torch.save(null_prompt, join(output_directory, "null_prompt.pt"))

In [ ]:
# Load Stable Diffusion
sd = StableDiffusion(device=get_device(), n_steps=25)
sd.quick_initialize().load_autoencoder(config.get_model("vae/vae")).load_decoder(config.get_model("vae/vae_decoder"))
sd.model.load_unet(config.get_model("unet/unet"))

In [ ]:
images = sd.generate_images_from_embeddings(null_prompt=null_prompt, embedded_prompt=embeddings, batch_size=batch_size)
get_memory_status(get_device())

In [ ]:
images.shape

In [ ]:
from stable_diffusion.utils_image import to_pil

if batch_size > 1:
    grid = torchvision.utils.make_grid(images, nrow=2, normalize=False, range=(-1, 1))
    img = to_pil(grid)
else:
    img = to_pil(images[0])
img

In [ ]:
images.shape

In [ ]:
img_encoder = CLIPImageEncoder(device=get_device())
get_memory_status(device=get_device())

In [ ]:
img_encoder.load_submodels()
get_memory_status(device=get_device())
img_encoder.initialize_preprocessor(do_center_crop=True)

In [ ]:
img_encoder.image_processor

In [ ]:
prep_from_img = img_encoder.preprocess_input(img).to(get_device())
type(img)

In [ ]:
img_encoder.image_processor

In [ ]:
prep_from_tensor = img_encoder.preprocess_input(images)
type(images)

In [ ]:
prep_from_img.shape, prep_from_tensor.shape

In [ ]:
torch.all(prep_from_img.to(get_device()) == prep_from_tensor)

In [ ]:
torch.norm(prep_from_img.to(get_device()) - prep_from_tensor)

In [ ]:
to_pil(prep_from_img.squeeze(0))

In [ ]:
to_pil(prep_from_tensor.squeeze(0))

In [ ]:
to_pil((prep_from_img - prep_from_tensor).squeeze(0))

In [ ]:
if batch_size > 1:
    grid = torchvision.utils.make_grid([prep_from_img, prep_from_tensor], nrow=2, normalize=False, range=(-1, 1))
    img = to_pil(grid)
else:
    img = to_pil(prep_from_img.squeeze(0))
img

In [47]:
import torchvision

grid = torchvision.utils.make_grid([prep_from_img.squeeze(), prep_from_tensor.squeeze()], nrow=2, normalize=False,
                                   range=(-1, 1))
img = to_pil(grid)